
# 📚 Literature Searching Toolkit (Academic Use Only)

Designed by **Dr. Chengkai Fan**  
Department of Mining, Metallurgical and Materials Engineering  
Université Laval

---

## 📌 Purpose
A simple GUI-based tool to perform literature searches using keywords via Google Scholar, visualizing year-wise trends and highlighting top-cited articles.

> ⚠ **Disclaimer**  
- This tool uses the `scholarly` library to query Google Scholar metadata.  
- It is **strictly for educational and personal use only**.  
- It **does not store full texts**, and **does not support commercial use**.


## 🧰 Import Required Libraries

In [ ]:

import tkinter as tk
from tkinter import scrolledtext, ttk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from scholarly import scholarly
from collections import Counter
import webbrowser
import time
import threading


## 🔧 Helper Functions

In [ ]:

def open_url(url):
    webbrowser.open(url)

def show_response_time(time_taken):
    progress_bar.stop()
    progress_label.config(text=f"Search Completed in {time_taken:.2f} seconds")

def show_top_cited(citation_list):
    top_cited = sorted(citation_list, key=lambda x: x[0], reverse=True)[:3]
    top_cited_display.delete(1.0, tk.END)
    top_cited_display.insert(tk.END, "Top 3 Most-Cited Papers:\n")
    for i, (citation_count, serial_number, title) in enumerate(top_cited, start=1):
        top_cited_display.insert(tk.END, f"{i}. Serial Number: {serial_number}, Title: {title}, Citations: {citation_count}\n")

def update_plot(years):
    for widget in plot_frame.winfo_children():
        widget.destroy()
    fig = Figure(figsize=(8, 6), dpi=100, facecolor='black')
    ax = fig.add_subplot(111)
    ax.set_facecolor('black')
    ax.spines['bottom'].set_color('white')
    ax.spines['top'].set_color('white')
    ax.spines['right'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.yaxis.label.set_color('white')
    ax.xaxis.label.set_color('white')
    ax.tick_params(axis='both', colors='white')
    year_counts = Counter(years)
    years_sorted = sorted(year_counts.keys())
    counts_sorted = [year_counts[year] for year in years_sorted]
    bars = ax.bar(years_sorted, counts_sorted, color='orange')
    ax.set_xlabel('Year', color='white')
    ax.set_ylabel('Number of Publications', color='white')
    ax.set_title('Publications per Year', color='white')
    canvas = FigureCanvasTkAgg(fig, master=plot_frame)
    canvas.draw()
    canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)


## 🔍 Search Logic and Display Function

In [ ]:

def search_and_display():
    start_time = time.time()
    keywords = [entry.get().strip() for entry in keyword_entries]
    keywords = [kw for kw in keywords if kw]
    if not keywords:
        result_display.insert(tk.END, "Please enter at least one keyword.\n")
        progress_bar.stop()
        return
    try:
        start_year = int(start_year_entry.get().strip())
        end_year = int(end_year_entry.get().strip())
    except ValueError:
        result_display.insert(tk.END, "Invalid year range.\n")
        progress_bar.stop()
        return
    if start_year > end_year:
        result_display.insert(tk.END, "Start year cannot be greater than end year.\n")
        progress_bar.stop()
        return
    review_only = review_only_var.get()
    results_to_fetch = int(results_count_combo.get())
    query = " ".join(keywords)
    search_query = None
    max_retries = 3
    for attempt in range(max_retries):
        try:
            search_query = scholarly.search_pubs(query)
            break
        except Exception as e:
            if attempt < max_retries - 1:
                result_display.insert(tk.END, f"Error: {e}. Retrying...\n")
                time.sleep(5)
            else:
                result_display.insert(tk.END, f"Error: {e}. All retries failed.\n")
                progress_bar.stop()
                return
    result_display.delete(1.0, tk.END)
    years = []
    citation_list = []
    results_fetched = 0
    serial_number = 1
    for i in range(results_to_fetch):
        try:
            publication = next(search_query)
            bib_info = publication.get('bib', {})
            pub_year = bib_info.get('pub_year', 'N/A')
            pub_abstract = bib_info.get('abstract', '')
            pub_url = publication.get('pub_url', 'No URL available')
            citation_count = publication.get('num_citations', 0)
            if pub_year.isdigit() and start_year <= int(pub_year) <= end_year:
                if review_only and "review" not in pub_abstract.lower():
                    continue
                result_display.insert(tk.END, f"{serial_number}. Title: {bib_info.get('title', 'No title')}\n")
                result_display.insert(tk.END, f"   Authors: {bib_info.get('author', 'No author')}\n")
                result_display.insert(tk.END, f"   Year: {pub_year}\n")
                result_display.insert(tk.END, f"   Citations: {citation_count}\n")
                url_tag = f"url{serial_number}"
                result_display.insert(tk.END, f"   URL: {pub_url}\n", url_tag)
                result_display.tag_config(url_tag, foreground="blue", underline=True)
                result_display.tag_bind(url_tag, "<Button-1>", lambda e, url=pub_url: open_url(url))
                result_display.insert(tk.END, "\n")
                years.append(pub_year)
                citation_list.append((citation_count, serial_number, bib_info.get('title', 'No title')))
                serial_number += 1
                results_fetched += 1
            progress_bar["value"] = (i / results_to_fetch) * 100
            root.update_idletasks()
            if results_fetched >= results_to_fetch:
                break
        except StopIteration:
            result_display.insert(tk.END, "No more results found.\n")
            break
        except Exception as e:
            result_display.insert(tk.END, f"An error occurred: {e}\n\n")
            continue
    update_plot(years)
    show_top_cited(citation_list)
    end_time = time.time()
    show_response_time(end_time - start_time)

def start_search():
    progress_bar.start()
    progress_label.config(text="Searching...")
    search_thread = threading.Thread(target=search_and_display)
    search_thread.start()


## 🖼️ GUI Layout and Main Application

In [ ]:

root = tk.Tk()
root.title("Literature Searching - Designed by Dr. Chengkai Fan at Université Laval")
root.configure(bg='black')

header_frame = tk.Frame(root, bg='#555555', height=50)
header_frame.pack(fill=tk.X)

keyword_frame = tk.Frame(root, bg='black')
keyword_frame.pack(pady=10)

label_frame = tk.Frame(keyword_frame, bg='black')
label_frame.pack()
tk.Label(label_frame, text="Enter Keywords:", bg='black', fg='white').pack(pady=5)

input_frame = tk.Frame(keyword_frame, bg='black')
input_frame.pack()

keyword_entries = []
for i in range(5):
    entry = tk.Entry(input_frame, width=25, bg='#90EE90', fg='black')
    entry.grid(row=0, column=i, padx=5)
    keyword_entries.append(entry)

options_frame = tk.Frame(root, bg='black')
options_frame.pack(pady=10)

tk.Label(options_frame, text="Start Year:", bg='black', fg='white').pack(side=tk.LEFT, padx=5)
start_year_entry = tk.Entry(options_frame, width=10)
start_year_entry.pack(side=tk.LEFT, padx=2)

tk.Label(options_frame, text="End Year:", bg='black', fg='white').pack(side=tk.LEFT, padx=5)
end_year_entry = tk.Entry(options_frame, width=10)
end_year_entry.pack(side=tk.LEFT, padx=2)

review_only_var = tk.BooleanVar()
review_only_checkbox = tk.Checkbutton(options_frame, text="Review Papers Only", variable=review_only_var, bg='black', fg='white', selectcolor='black')
review_only_checkbox.pack(side=tk.LEFT, padx=5)

tk.Label(options_frame, text="Results:", bg='black', fg='white').pack(side=tk.LEFT, padx=5)
results_count_combo = ttk.Combobox(options_frame, values=[10, 20, 30, 40, 50, 100], width=7)
results_count_combo.set(10)
results_count_combo.pack(side=tk.LEFT, padx=5)

search_button = tk.Button(root, text="Search", command=start_search)
search_button.pack(pady=5)

progress_bar = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=400, mode='indeterminate')
progress_bar.pack(pady=5)

progress_label = tk.Label(root, text="Ready")
progress_label.pack(pady=5)

paned_window = tk.PanedWindow(root, orient=tk.HORIZONTAL)
paned_window.pack(fill=tk.BOTH, expand=True)

result_frame = tk.Frame(paned_window, bg='black')
top_cited_frame = tk.Frame(paned_window, bg='black')
paned_window.add(result_frame, stretch="always")
paned_window.add(top_cited_frame, stretch="always")

result_display = scrolledtext.ScrolledText(result_frame, wrap=tk.WORD, width=80, height=15)
result_display.pack(pady=5)

top_cited_display = scrolledtext.ScrolledText(top_cited_frame, wrap=tk.WORD, width=60, height=15)
top_cited_display.pack(pady=5)

plot_frame = tk.Frame(root)
plot_frame.pack(fill=tk.BOTH, expand=True)

root.mainloop()
